### 加载模型和数据集

In [1]:
import torch

print(torch.cuda.is_available())

from modelscope import AutoTokenizer
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq

tokenizer = AutoTokenizer.from_pretrained("./Qwen2.5-0.5B-Instruct", use_fast=False, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("./Qwen2.5-0.5B-Instruct", device_map="cuda", torch_dtype="auto")

True


d:\Programs\anaconda3\envs\cuda121-2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd
from datasets import Dataset
from swanlab.integration.huggingface import SwanLabCallback
from peft import LoraConfig, TaskType, get_peft_model
import swanlab

C:\Users\29277\AppData\Local\Temp\ipykernel_12944\2019269277.py:3: DeprecationWarning: The module 'huggingface' is deprecated and will be removed in future versions. Please update your imports to use 'transformers' instead.
  from swanlab.integration.huggingface import SwanLabCallback


In [3]:
train_df = pd.read_json('multi-class-large.json')
train_ds = Dataset.from_pandas(train_df)

In [4]:
train_df

,文本,风险类别
0,您好，米丽转女士，我们是快手网店的客服，需要收取您的代销商代理费。请您下载“中国建设银行”A...,冒充电商物流客服类
1,王总让我通知您，公司下周将有一场重要的产品发布会，需要您提前报名参加。请点击以下链接完成报名...,冒充领导、熟人类
2,我是您的老朋友张华，最近我们公司需要更新一批电子设备，我想请您帮忙处理一下。请您尽快登录我们...,冒充领导、熟人类
3,您好，抖音点赞任务升级啦！现在只需下载“翎信”软件，即可参与大额任务，收益翻倍！点击链接下载...,虚假网络投资理财类
4,您好，我是部队采购部门的，现需大量自热饭，请您通过我们推荐的厂家进行采购，详情请联系手机号1...,冒充军警购物类诈骗
...,...,...
39267,您好，我是支付宝客服，您的账户需要更新为成年用户，否则将影响您的个人征信。请通过QQ投屏操作...,虚假信用服务类
39268,尊敬的球迷，由于系统升级，您的会员资料需要更新。请点击以下链接更新您的个人信息，以便继续享受...,冒充领导、熟人类
39269,您好，我是鼎晖资本的助教，近期我们有一个投资机会在CDH APP上，现邀请您加入我们的投资群...,虚假网络投资理财类
39270,您好，我是您在问道游戏中的好友‘知足常乐’，如果您考虑出售游戏账号，建议您通过官方渠道进行交...,无风险


In [5]:
s = [(len(i),i) for i in list(train_ds['文本'])]
s.sort(reverse=True)

In [6]:
s

[(444,
  '你好，我是曲靖市公安局的民警。我打电话是想通知你一件事情：我们在调查一起涉及洗黑钱的案件时发现你的银行卡尾号是2184也被牵涉其中。现在这个案子已经移交给东城分局处理了。我们需要进一步核实你和这起案件的关系以及相关的信息资料和证据材料等是否真实合法合规等等问题首先请你添加我的QQ号码吧！然后我们可以通过视频通话来确认彼此的身份信息和相关情况；接着再给你一个链接查询你是否涉及到犯罪团伙洗钱一案的信息并让你填写个人信息及家庭成员和社会关系等情况以备查证……最终要告诉你一件事:由于此案情重大复杂如果要证明自己清白无辜就必须按照我们的要求进行配合工作——必须将自己的流动资产转到指定的银保监会的安全账号里核定其财产状况是否有异常现象发生…最后还要下载一款名叫ZOOM的手机APP软件并且加入到一个会议里面协助我们来控制共享屏幕监督着你以便于对你审查冤枉错判等现象出现我们会根据所掌握的相关线索和你本人实际情况而慎重作出公正判决希望你能够积极响应号召主动交代事实真相争取宽大处理重新做人的机会'),
 (411,
  '您好！我是公安局的工作人员。我们发现您涉嫌非法出境活动，需要您的配合调查。请务必保持冷静并按照我们的指示操作。我们已经获取了相关证据并且正在与瞩目的app进行联系...请您不要挂断电话或关闭通话窗口以免影响后续的操作流程。我们会尽快完成核实工作并与你取得进一步的沟通交流以便了解情况并提供帮助和支持......请不要担心也不要轻信他人或者自行采取行动避免造成不必要的损失或者其他不良后果的发生，。如果有任何疑问或其他问题可以随时与我们联络以获得更多的信息支持和服务保障……现在我们需要进一步核对相关信息并进行转账处理等事宜的具体说明…，希望这不会给您带来困扰或不安全因素的产生.我会在事情结束后立即将资金归还到原来的账户中，同时也会积极协助警方开展相关工作以确保自身权益不受损害的同时也维护社会秩序的正常运行....请相信公安机关的工作能力和公正执法态度.....再次感谢您的理解和合作!现在就让我们一起解决这个问题吧~"'),
 (404,
  '你好！我是南京市公安局刑侦大队的。我接到了一起虐待儿童案件的相关信息，有人使用您的身份证为一起这样的案子办理住院相关手续了。我们正在调查这个事情并需要您配合我们的工作进行一些核实和确认情况的工作流程如下:鉴于目前疫

In [3]:
category = "短信风险类别选项有：无风险|冒充电商物流客服类|虚假网络投资理财类|虚假信用服务类|虚假购物、服务类|冒充公检法及政府机关类|冒充领导、熟人类|网络婚恋、交友类|冒充军警购物类诈骗|网黑案件"
prompt = '''在这个任务中，你是一位资深的反诈骗网络安全分析师，你的职责是利用你的专业知识和对网络诈骗行为的深刻理解，从短信文本中识别出可能存在的欺诈行为和风险类别。你的工作对于提前预警潜在的网络诈骗，保护用户财产安全和个人信息不被侵犯具有重要意义。现在，请仔细审查以下短信文本，并运用你的专业判断，给出短信的风险类别判断结果。(%s)'''%category

def process_func(example):
    """
    将数据集进行预处理
    """
    MAX_LENGTH = 384 
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer(
        f"<|im_start|>system\n{prompt}<|im_end|>\
        \n<|im_start|>user\n{example['文本']}<|im_end|>\n<|im_start|>assistant\n",
        add_special_tokens=False,
    )
    response = tokenizer(f"{example['风险类别']}", add_special_tokens=False)
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    attention_mask = (
        instruction["attention_mask"] + response["attention_mask"] + [1]
    )
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]
    if len(input_ids) > MAX_LENGTH:  # 做一个截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}  


In [8]:
train_dataset = train_ds.map(process_func, remove_columns=train_ds.column_names)
train_dataset.save_to_disk("multi-class-large")

d:\Programs\anaconda3\envs\cuda121-2\Lib\site-packages\datasets\utils\_dill.py:385: DeprecationWarning: co_lnotab is deprecated, use co_lines instead.
  obj.co_lnotab,  # for < python 3.10 [not counted in args]
Saving the dataset (1/1 shards): 100%|██████████| 39272/39272 [00:00<00:00, 765894.85 examples/s]


In [4]:
from datasets import load_from_disk
train_dataset = load_from_disk("multi-class-large")

In [5]:
train_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 39272
})

### 模型微调

In [6]:
model = AutoModelForCausalLM.from_pretrained("./Qwen2.5-0.5B-Instruct", device_map="auto", torch_dtype="auto")
config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj", "lm_head"],
    inference_mode=False,  # 训练模式
    r=8,  # Lora 秩
    lora_alpha=32,  # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1,  # Dropout 比例
)

# for param in model.parameters():
#     param.requires_grad = True
    
model = get_peft_model(model, config)

In [12]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 896)
        (layers): ModuleList(
          (0-23): 24 x Qwen2DecoderLayer(
            (self_attn): Qwen2SdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=896, out_features=896, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=896, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=896, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(
   

In [7]:
model.print_trainable_parameters()

trainable params: 5,621,760 || all params: 499,654,528 || trainable%: 1.1251


In [8]:
args = TrainingArguments(
    output_dir="./output/multi-class-large0116",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    logging_steps=1000,
    num_train_epochs=4,
    save_steps=1000,
    learning_rate=1e-4,
    save_on_each_node=True,
    gradient_checkpointing=True,
    report_to="none",
)

swanlab_callback = SwanLabCallback(
    project="Qwen2.5-0.5B-Instruct-Finetuning",
    experiment_name="Qwen2.5-0.5B-Instruct-multi-class-large0116",
    description="Qwen2.5-0.5B-Instruct模型在multi-class-large数据集上微调。",
    config={
        "model": "qwen/Qwen2.5-0.5B-Instruct",
        "dataset": "FGRC-SCD电信诈骗数据集 multi-class-large",
    }
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
    callbacks=[swanlab_callback],
)

In [9]:
trainer.train()

swanlab: swanlab version 0.4.2 is available!  Upgrade: `pip install -U swanlab`
swanlab: \ Getting project...                                                      

d:\Programs\anaconda3\envs\cuda121-2\Lib\site-packages\swanlab\api\http.py:114: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  if (self.sid_expired_at - datetime.utcnow()).total_seconds() <= self.REFRESH_TIME:


swanlab: Tracking run with swanlab version 0.3.21                                  
swanlab: Run data will be saved locally in d:\fudan\LLM\FineTune-Qwen\swanlog\run-20250116_001958-a3b1799d
swanlab: 👋 Hi tangerine, welcome to swanlab!
swanlab: Syncing run Qwen2.5-0.5B-Instruct-multi-class-large0116 to the cloud
swanlab: 🌟 Run `swanlab watch d:\fudan\LLM\FineTune-Qwen\swanlog` to view SwanLab Experiment Dashboard locally
swanlab: 🏠 View project at https://swanlab.cn/@tangerine/Qwen2.5-0.5B-Instruct-Finetuning
swanlab: 🚀 View run at https://swanlab.cn/@tangerine/Qwen2.5-0.5B-Instruct-Finetuning/runs/nlx11qnu3rvoxzie8c5uw


  0%|          | 0/9816 [00:00<?, ?it/s]`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
d:\Programs\anaconda3\envs\cuda121-2\Lib\site-packages\torch\_dynamo\eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
d:\Programs\anaconda3\envs\cuda121-2\Lib\site-packages\torch\utils\checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
d:\Programs\anaconda3\envs\cuda121-2\Lib\site-packages\transformers\models\qwen2\modeling_qwen2.py:580: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-

{'loss': 0.4303, 'grad_norm': 11.320490837097168, 'learning_rate': 8.981255093724531e-05, 'epoch': 0.41}


...

{'loss': 0.1356, 'grad_norm': 3.0932700634002686, 'learning_rate': 8.312958435207824e-06, 'epoch': 3.67}


d:\Programs\anaconda3\envs\cuda121-2\Lib\site-packages\torch\_dynamo\eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
d:\Programs\anaconda3\envs\cuda121-2\Lib\site-packages\torch\utils\checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
d:\Programs\anaconda3\envs\cuda121-2\Lib\site-packages\swanlab\api\http.py:114: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  if (self.sid_expired_at - datetime.utcnow()).

{'train_runtime': 7178.78, 'train_samples_per_second': 21.882, 'train_steps_per_second': 1.367, 'train_loss': 0.18228594073456483, 'epoch': 4.0}


TrainOutput(global_step=9816, training_loss=0.18228594073456483, metrics={'train_runtime': 7178.78, 'train_samples_per_second': 21.882, 'train_steps_per_second': 1.367, 'total_flos': 8.269148222023066e+16, 'train_loss': 0.18228594073456483, 'epoch': 3.9991851700957426})

d:\Programs\anaconda3\envs\cuda121-2\Lib\site-packages\swanlab\api\http.py:114: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  if (self.sid_expired_at - datetime.utcnow()).total_seconds() <= self.REFRESH_TIME:


In [16]:
swanlab.finish()

swanlab: 🌟 Run `swanlab watch d:\fudan\LLM\FineTune-Qwen\swanlog` to view SwanLab Experiment Dashboard locally
swanlab: 🏠 View project at https://swanlab.cn/@tangerine/Qwen2.5-0.5B-Instruct-Finetuning
swanlab: 🚀 View run at https://swanlab.cn/@tangerine/Qwen2.5-0.5B-Instruct-Finetuning/runs/tkdo88q8pune5mb3xypvr
